import library

In [1]:
import torch
import numpy as np
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification
from scipy.io.wavfile import read

config

In [2]:
audio_path = ["../data/1-17-2017/0.wav", "../data/1-17-2017/1.wav", "../data/1-17-2017/2.wav", "../data/1-17-2017/3.wav", "../data/1-17-2017/4.wav", "../data/1-17-2017/5.wav"]
#audio_path = ["../data/1-17-2017/5.wav"]

modelw = Wav2Vec2ForSequenceClassification.from_pretrained('alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech')
processor = Wav2Vec2FeatureExtractor.from_pretrained('alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

label2id = {
    "female": 0,
    "male": 1
}

id2label = {
    0: "female",
    1: "male"
}

num_labels = 2

inference

In [3]:
results_dict = {}

def store_results(prob, file_name):
  prob_list = prob.tolist()[0]
  if prob_list[1] > prob_list[0]:
    gender = "Female"
    probability = prob_list[1]
  else:
    gender = "Male"
    probability = prob_list[0]
  results_dict[file_name] = {'pred': gender, 'prob': probability}

for path in audio_path:
  output = read(path)
  waveform = np.array(output[1],dtype=float)
  sound_array = np.array(waveform)
  input_values = processor(sound_array, sampling_rate = 16000, padding='longest', return_tensors='pt').input_values
  with torch.no_grad():
    result = modelw(input_values).logits
    prob = torch.nn.functional.softmax(result, dim=1)
    store_results(prob, path)

results

In [4]:
results_dict

{'../data/1-17-2017/0.wav': {'pred': 'Female', 'prob': 0.998759388923645},
 '../data/1-17-2017/1.wav': {'pred': 'Female', 'prob': 0.9986407160758972},
 '../data/1-17-2017/2.wav': {'pred': 'Male', 'prob': 0.9919637441635132},
 '../data/1-17-2017/3.wav': {'pred': 'Female', 'prob': 0.9970661997795105},
 '../data/1-17-2017/4.wav': {'pred': 'Male', 'prob': 0.9986563920974731},
 '../data/1-17-2017/5.wav': {'pred': 'Male', 'prob': 0.99853515625}}

In [5]:
#true labels: ["Male", "Female", "Male", "Male", "Male", "Female"]
#{'../data/1-17-2017/0.wav': {'pred': 'Female', 'prob': 0.998759388923645}}
#{'../data/1-17-2017/1.wav': {'pred': 'Female', 'prob': 0.9986407160758972}}
#{'../data/1-17-2017/2.wav': {'pred': 'Male', 'prob': 0.9919637441635132}}
#{'../data/1-17-2017/3.wav': {'pred': 'Female', 'prob': 0.9970661997795105}}
#{'../data/1-17-2017/4.wav': {'pred': 'Male', 'prob': 0.9986541271209717}}
#{'../data/1-17-2017/5.wav': {'pred': 'Male', 'prob': 0.99853515625}}

In [6]:
prob

tensor([[0.9985, 0.0015]])